<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Schema_Voto_Geologia_OverrideManuale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Schema del voto finale — Geologia (Override manuale Rocce/Carte prioritario)

**Novità**: se spunti **"Inserisci manuale"** per *Rocce* o *Carte*, il valore **manuale** ha **priorità** e **sovrascrive** quello del menu a tendina, indipendentemente dalla selezione corrente. Il dropdown viene disabilitato quando è attivo l'inserimento manuale, per evitare ambiguità.

Resto del flusso invariato:
1) Carica Excel con **Cognome**, **Nome**, **Matricola**, **Rocce**, **Carte**.  
2) Cerca per **Cognome Nome** → visualizza **tutti gli appelli**.  
3) Seleziona l’appello e scegli **Rocce/Carte** dai numerici estratti **oppure** spunta **Inserisci manuale** e digita i valori.  
4) Inserisci **VS** e calcola secondo il protocollo (emergenza a 18, media pesata con HALF‑DOWN, caso 30 e lode).


In [3]:

import io, re
import pandas as pd
from decimal import Decimal, ROUND_HALF_DOWN

import ipywidgets as widgets
from IPython.display import display, HTML

# ------------------------
# Utility
# ------------------------

def normalize_colnames(df):
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def find_column(df, candidates):
    cols = list(df.columns)
    for cand in candidates:
        cand_l = cand.lower()
        for c in cols:
            if c == cand_l:
                return c
    return None

def build_key(cognome, nome):
    def norm(x):
        return str(x).strip().upper().replace("  ", " ")
    return f"{norm(cognome)} {norm(nome)}".strip()

def extract_numeric_candidates(val):
    out = []
    if pd.isna(val):
        return out
    try:
        fv = float(val)
        if 0 <= fv <= 30:
            return [fv]
    except:
        pass
    s = str(val)
    tokens = re.findall(r'(?<!\d)(\d{1,2}(?:[.,]\d+)?|30(?:[.,]0+)?)', s)
    for t in tokens:
        t = t.replace(',', '.')
        try:
            fv = float(t)
            if 0 <= fv <= 30:
                out.append(fv)
        except:
            continue
    out = sorted(set(out))
    return out

def round_half_down(x):
    d = Decimal(str(x)).quantize(Decimal('1'), rounding=ROUND_HALF_DOWN)
    return int(d)

def compute_vp(rocce, carte):
    return (rocce + carte) / 2.0

def compute_final(vp, vs):
    if vs == 18:
        return "18", "voto di emergenza", float(18)
    if int(round(vp)) == 30 and vs == 30:
        return "30 e lode", "media pesata + bonus", 30.0
    weighted = (vp * 2.0 + vs * 3.0) / 5.0
    finale_int = round_half_down(weighted)
    return str(finale_int), "media pesata", float(weighted)

# ------------------------
# Widget
# ------------------------

uploader = widgets.FileUpload(accept='.xlsx,.xls', multiple=False, description='Carica Excel')
search_name = widgets.Text(value='', description='Cognome Nome:', placeholder='es. Rossi Maria', layout=widgets.Layout(width='50%'))
search_button = widgets.Button(description='Cerca', icon='search')
row_select = widgets.Dropdown(options=[], description='Appello:', layout=widgets.Layout(width='60%'))
matricola_label = widgets.HTML("<b>Matricola:</b> —")

rocce_dd = widgets.Dropdown(options=[], description='Rocce:', layout=widgets.Layout(width='40%'))
rocce_manual = widgets.BoundedFloatText(value=18.0, min=0.0, max=30.0, step=0.5, description='Rocce (man.):', layout=widgets.Layout(width='40%'))
rocce_use_manual = widgets.Checkbox(value=False, description='Inserisci manuale')

carte_dd = widgets.Dropdown(options=[], description='Carte:', layout=widgets.Layout(width='40%'))
carte_manual = widgets.BoundedFloatText(value=18.0, min=0.0, max=30.0, step=0.5, description='Carte (man.):', layout=widgets.Layout(width='40%'))
carte_use_manual = widgets.Checkbox(value=False, description='Inserisci manuale')

vs_input = widgets.BoundedIntText(value=18, min=18, max=30, step=1, description='VS (scritta):')
calc_button = widgets.Button(description='Calcola voto finale', button_style='primary', icon='calculator')

out_info = widgets.Output()
out_table = widgets.Output()
out_result = widgets.Output()

display(HTML("<h3>1) Carica l’Excel degli studenti</h3>"))
display(uploader)
display(HTML("<h3 style='margin-top:20px'>2) Cerca studente</h3>"))
display(widgets.HBox([search_name, search_button]))
display(out_info, out_table)

display(HTML("<h3 style='margin-top:20px'>3) Seleziona appello e valori Rocce/Carte</h3>"))
display(widgets.HBox([row_select, matricola_label]))
display(widgets.HBox([rocce_dd, rocce_use_manual, rocce_manual]))
display(widgets.HBox([carte_dd, carte_use_manual, carte_manual]))

display(HTML("<h3 style='margin-top:20px'>4) Inserisci VS e calcola</h3>"))
display(widgets.HBox([vs_input, calc_button]))
display(out_result)

# ------------------------
# Stato
# ------------------------
import pandas as pd
df_global = None
key_col = "_key_"
colmap = {}
hits_df = pd.DataFrame()

# ------------------------
# Callbacks
# ------------------------

def on_upload_change(change):
    global df_global, colmap
    out_info.clear_output(); out_table.clear_output(); out_result.clear_output()
    row_select.options = []
    rocce_dd.options = []; carte_dd.options = []
    matricola_label.value = "<b>Matricola:</b> —"
    if len(uploader.value) == 0:
        return
    fileinfo = next(iter(uploader.value.values()))
    content = fileinfo['content']
    try:
        df = pd.read_excel(io.BytesIO(content))
    except Exception as e:
        with out_info:
            print("Errore nella lettura del file Excel:", e)
        return
    df = normalize_colnames(df)
    c_cognome = find_column(df, ['cognome','surname','last','last name'])
    c_nome    = find_column(df, ['nome','name','first','first name'])
    c_matr    = find_column(df, ['matricola','id','matr','student id','studentid'])
    c_rocce   = find_column(df, ['rocce','rock','rocks','prova rocce','voto rocce','rocce voto'])
    c_carte   = find_column(df, ['carte','mappe','carte geologiche','geologic maps','prova carte','voto carte','carte voto'])
    missing = []
    for lab, col in [('Cognome', c_cognome), ('Nome', c_nome), ('Matricola', c_matr), ('Rocce', c_rocce), ('Carte', c_carte)]:
        if col is None:
            missing.append(lab)
    if missing:
        with out_info:
            print("Colonne mancanti nell’Excel:", ", ".join(missing))
            print("Colonne viste:", list(df.columns))
            print("Rinomina/aggiusta le colonne come: Cognome, Nome, Matricola, Rocce, Carte (case-insensitive).")
        return
    df[key_col] = [build_key(row[c_cognome], row[c_nome]) for _, row in df.iterrows()]
    colmap.update({'cognome': c_cognome, 'nome': c_nome, 'matricola': c_matr, 'rocce': c_rocce, 'carte': c_carte})
    df_global = df
    with out_info:
        print(f"File caricato: {len(df)} righe.")
        display(df[[c_cognome, c_nome, c_matr, c_rocce, c_carte]].head(5))

uploader.observe(on_upload_change, names='value')

def refresh_row_options(df_hits):
    opts = []
    for idx, r in df_hits.iterrows():
        cogn = str(r[colmap['cognome']]); nome = str(r[colmap['nome']])
        matr = str(r[colmap['matricola']]); roc  = str(r[colmap['rocce']]); car  = str(r[colmap['carte']])
        label = f"idx={idx} | {cogn} {nome} | Matricola {matr} | Rocce='{roc}' | Carte='{car}'"
        opts.append((label, idx))
    row_select.options = opts
    if opts:
        row_select.value = opts[0][1]

def fill_rc_dropdowns_from_row(row):
    roc_vals = extract_numeric_candidates(row[colmap['rocce']])
    car_vals = extract_numeric_candidates(row[colmap['carte']])
    roc_opts = [(f"{v:.2f}", v) for v in roc_vals] if roc_vals else [("— nessun valore numerico —", None)]
    car_opts = [(f"{v:.2f}", v) for v in car_vals] if car_vals else [("— nessun valore numerico —", None)]
    rocce_dd.options = roc_opts; carte_dd.options = car_opts
    rocce_dd.value = roc_opts[0][1]; carte_dd.value = car_opts[0][1]

def on_search_clicked(b):
    global hits_df
    out_info.clear_output(); out_table.clear_output(); out_result.clear_output()
    rocce_dd.options = []; carte_dd.options = []
    matricola_label.value = "<b>Matricola:</b> —"
    if df_global is None:
        with out_info:
            print("Caricare prima l’Excel.")
        return
    q = search_name.value.strip().upper()
    if not q:
        with out_info:
            print("Inserire 'Cognome Nome' (es. Rossi Maria).")
        return
    hits_df = df_global[df_global[key_col].str.contains(q, na=False, regex=False)].copy()
    if hits_df.empty:
        with out_info:
            print(f"Nessuno studente trovato per '{search_name.value}'.")
        row_select.options = []
        return
    with out_table:
        display(hits_df[[colmap['cognome'], colmap['nome'], colmap['matricola'], colmap['rocce'], colmap['carte']]].reset_index().rename(columns={'index':'idx'}))
    refresh_row_options(hits_df)
    sel_idx = row_select.value
    row = hits_df.loc[sel_idx]
    fill_rc_dropdowns_from_row(row)
    matricola_label.value = f"<b>Matricola:</b> {row[colmap['matricola']]}"

search_button.on_click(on_search_clicked)

def on_row_change(change):
    out_result.clear_output()
    if change['name'] != 'value' or hits_df is None or hits_df.empty:
        return
    sel_idx = change['new']
    if sel_idx is None:
        return
    row = hits_df.loc[sel_idx]
    fill_rc_dropdowns_from_row(row)
    matricola_label.value = f"<b>Matricola:</b> {row[colmap['matricola']]}"

row_select.observe(on_row_change, names='value')

def toggle_manual_widgets(*args):
    rocce_dd.disabled = rocce_use_manual.value
    rocce_manual.disabled = not rocce_use_manual.value
    carte_dd.disabled = carte_use_manual.value
    carte_manual.disabled = not carte_use_manual.value

rocce_use_manual.observe(toggle_manual_widgets, names='value')
carte_use_manual.observe(toggle_manual_widgets, names='value')
toggle_manual_widgets()

def get_selected_value(drop, use_manual_chk, manual_widget):
    """
    PRIORITÀ MANUALE:
    - Se la checkbox 'Inserisci manuale' è attiva -> restituisci SEMPRE manual_widget.value
    - Altrimenti usa il valore del dropdown, se presente
    """
    if use_manual_chk.value:
        return float(manual_widget.value)
    val = drop.value
    if val is not None:
        return float(val)
    raise ValueError("Nessun valore disponibile: attivare 'Inserisci manuale' oppure selezionare un'opzione dal menu.")

def on_calculate_clicked(b):
    out_result.clear_output()
    if hits_df is None or hits_df.empty:
        with out_result:
            print("Eseguire prima la ricerca e selezionare un appello.")
        return
    try:
        v_rocce = get_selected_value(rocce_dd, rocce_use_manual, rocce_manual)
        v_carte = get_selected_value(carte_dd, carte_use_manual, carte_manual)
    except Exception as e:
        with out_result:
            print(str(e))
        return
    for v, lab in [(v_rocce,'Rocce'), (v_carte,'Carte')]:
        if not (0.0 <= v <= 30.0):
            with out_result:
                print(f"Valore fuori range per {lab}: {v}. Atteso 0..30.")
            return
    vp = compute_vp(v_rocce, v_carte)
    vs = int(vs_input.value)
    finale_str, metodo, weighted = compute_final(vp, vs)
    sel_idx = row_select.value
    row = hits_df.loc[sel_idx]
    with out_result:
        html = f"""
        <div style="font-family: ui-monospace, Consolas, monospace; line-height:1.5; font-size:15px">
            <div><b>Studente:</b> {row[colmap['cognome']]} {row[colmap['nome']]} — <b>Matricola:</b> {row[colmap['matricola']]}</div>
            <div><b>Rocce (selez./man.):</b> {v_rocce:.2f} — <b>Carte (selez./man.):</b> {v_carte:.2f} — <b>VP (media):</b> {vp:.2f}</div>
            <div><b>VS (scritta):</b> {vs:d}</div>
            <hr style="border:none;border-top:1px solid #ccc;"/>
            <div><b>Voto pesato:</b> (VP×2 + VS×3)/5 = {weighted:.2f}</div>
            <div><b>Metodo:</b> {metodo}</div>
            <div style="font-size:18px"><b>VOTO FINALE:</b> {finale_str}</div>
        </div>
        """
        display(HTML(html))

calc_button.on_click(on_calculate_clicked)


FileUpload(value={}, accept='.xlsx,.xls', description='Carica Excel')

Output()

Output()

Output()